In [ ]:
# If running from Colab, please uncomment and run the following cell to get summer installed.
#!pip install summerepi2==1.3.5

In [ ]:
import pandas as pd
from datetime import datetime
from summer2 import CompartmentalModel, Stratification
from summer2.parameters import Parameter
from plotly import graph_objects as go
pd.options.plotting.backend = 'plotly'

## Stratification with reduced susceptiblity

### Stratification
This notebook applies a simple two-strata stratification 
to the basic model introduced in the SEIR model notebook.
The two strata are arbitrarily referred to as child and adult
implying that they are intended to represent age stratification,
but this could easily represent other population heterogeneities.

### Susceptibility modification
After creating the stratification object,
we modify the relative susceptibilty of the "child" stratum.
Here we allow for children to be half as susceptible as adults,
by modifying the infection flow for children
(that is, they get infected at half the rate 
that they would have been infected if the modification had 
not been applied - implying that they are half as 
susceptible to infection).

More generally, this code is intended to represent 
the way in which users can modify parameter values 
for subpopulations of the model.

To understand the effect on each population sub-group
we will need to track a stratum-specific output.

In [ ]:
iso = 'MYS'

In [ ]:
cases_data = pd.read_csv('https://github.com/monash-emu/wpro_working/raw/main/data/new_cases.csv', index_col=0)[iso]
cases_data.index = pd.to_datetime(cases_data.index)
approx_pops = {
    'MYS': 33e6,
    'PHL': 114e6,
    'VNM': 97e6,
}

In [ ]:
analysis_start_date = datetime(2021, 8, 1) 
analysis_end_date = datetime(2022, 6, 1)

epi_model = CompartmentalModel(
    [analysis_start_date, analysis_end_date],
    ['susceptible', 'exposed', 'infectious', 'recovered'],
    ['infectious'],
    ref_date=datetime(2019, 12, 31),
)
epi_model.add_infection_frequency_flow('infection', Parameter('contact_rate'), 'susceptible', 'exposed')
epi_model.add_transition_flow('progression', 1.0 / Parameter('incubation_period'), 'exposed', 'infectious')
epi_model.add_transition_flow('recovery', 1.0 / Parameter('infectious_period'), 'infectious', 'recovered')
epi_model.set_initial_population({'susceptible': approx_pops[iso], 'infectious': 1.0})

In [ ]:
# Stratification
age_strata = ['child', 'adult']
age_strat = Stratification('age', age_strata, epi_model._original_compartment_names)
age_suscept = {'child': Parameter('child_rel_suscept'), 'adult': None}
age_strat.set_flow_adjustments('infection', age_suscept)
epi_model.stratify_with(age_strat)
for strat in age_strata:
    inc_string = f'incX{strat}'
    age_inc = epi_model.request_output_for_flow(inc_string, 'progression', source_strata={'age': strat}, save_results=False)
    epi_model.request_function_output(f'notifX{strat}', age_inc * Parameter('detection_prop'))

In [ ]:
parameters = {
    'contact_rate': 0.5,
    'incubation_period': 5.0,
    'infectious_period': 5.0,
    'detection_prop': 0.07,
    'child_rel_suscept': 0.5,
}
epi_model.run(parameters)

In [ ]:
plot_start_date = datetime(2021, 10, 1)
plot = epi_model.get_derived_outputs_df()[[f'notifX{strat}' for strat in age_strata]].plot.area()
plot.update_xaxes(range=(plot_start_date, analysis_end_date))
plot.add_trace(go.Scatter(x=cases_data.index, y=cases_data, mode='markers', name='data'))